In [2]:
from llmops.text_utils import TextFileLoader, CharacterTextSplitter
from llmops.vectordatabase import VectorDatabase
import asyncio
from llmops.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)
from llmops.retrieval_pipeline import RetrievalAugmentedQAPipeline, WandB_RetrievalAugmentedQAPipeline
from llmops.openai_utils.chatmodel import ChatOpenAI
import wandb
import datetime


In [3]:
import nest_asyncio
nest_asyncio.apply()

## Loading the data

In [4]:
text_loader = TextFileLoader("data/KingLear.txt")
documents = text_loader.load_documents()
len(documents)

1

In [5]:
print(documents[0][:100])

ACT I
SCENE I. King Lear's palace.
Enter KENT, GLOUCESTER, and EDMUND
KENT
I thought the king had m


## Splitting Data into Chunks

In [6]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

189

In [7]:
split_documents[0:1]

["\ufeffACT I\nSCENE I. King Lear's palace.\nEnter KENT, GLOUCESTER, and EDMUND\nKENT\nI thought the king had more affected the Duke of\nAlbany than Cornwall.\nGLOUCESTER\nIt did always seem so to us: but now, in the\ndivision of the kingdom, it appears not which of\nthe dukes he values most; for equalities are so\nweighed, that curiosity in neither can make choice\nof either's moiety.\nKENT\nIs not this your son, my lord?\nGLOUCESTER\nHis breeding, sir, hath been at my charge: I have\nso often blushed to acknowledge him, that now I am\nbrazed to it.\nKENT\nI cannot conceive you.\nGLOUCESTER\nSir, this young fellow's mother could: whereupon\nshe grew round-wombed, and had, indeed, sir, a son\nfor her cradle ere she had a husband for her bed.\nDo you smell a fault?\nKENT\nI cannot wish the fault undone, the issue of it\nbeing so proper.\nGLOUCESTER\nBut I have, sir, a son by order of law, some year\nelder than this, who yet is no dearer in my account:\nthough this knave came something s

## Adding Embeddings and Vectors

In [8]:
from dotenv import load_dotenv
import os
import openai
load_dotenv()

True

In [9]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

In [10]:
vector_db.search_by_text("Your servant Kent. Where is your servant Caius?", k=3)

[("ng] O my good master!\nKING LEAR\nPrithee, away.\nEDGAR\n'Tis noble Kent, your friend.\nKING LEAR\nA plague upon you, murderers, traitors all!\nI might have saved her; now she's gone for ever!\nCordelia, Cordelia! stay a little. Ha!\nWhat is't thou say'st? Her voice was ever soft,\nGentle, and low, an excellent thing in woman.\nI kill'd the slave that was a-hanging thee.\nCaptain\n'Tis true, my lords, he did.\nKING LEAR\nDid I not, fellow?\nI have seen the day, with my good biting falchion\nI would have made them skip: I am old now,\nAnd these same crosses spoil me. Who are you?\nMine eyes are not o' the best: I'll tell you straight.\nKENT\nIf fortune brag of two she loved and hated,\nOne of them we behold.\nKING LEAR\nThis is a dull sight. Are you not Kent?\nKENT\nThe same,\nYour servant Kent: Where is your servant Caius?\nKING LEAR\nHe's a good fellow, I can tell you that;\nHe'll strike, and quickly too: he's dead and rotten.\nKENT\nNo, my good lord; I am the very man,--\nKING LEA

## Creating and Prompting OpenAI

In [11]:
chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
    system_role_prompt.create_message(expertise="Python"),
]

response = chat_openai.run(messages)

In [12]:
print(response)

The best way to write a loop depends on the specific task and programming language you are using. However, I can provide you with some general tips that can help you write efficient and effective loops:

1. Choose the appropriate loop construct: Different programming languages offer various loop constructs, such as "for" loops, "while" loops, or "do-while" loops. Understand the requirements of your task and select the loop construct that best suits your needs.

2. Define the loop conditions: Clearly specify the conditions that determine when the loop should continue running and when it should terminate. This will help avoid infinite loops and ensure the loop runs for the necessary iterations.

3. Initialize loop variables: Initialize any loop variables or counters before entering the loop. It ensures that the loop starts with the correct initial values.

4. Update loop variables: Update loop variables within the loop body based on the task requirements. This could involve incrementing 

## Retrieval Augmented Question Answering Prompt

In [13]:
RAQA_PROMPT_TEMPLATE = """
Use the provided context to answer the user's query. 

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".

Context:
{context}
"""

raqa_prompt = SystemRolePrompt(RAQA_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """
User Query:
{user_query}
"""

user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)



In [14]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

In [15]:
retrieval_augmented_qa_pipeline.run_pipeline("Who is King Lear?", raqa_prompt, user_prompt)

'King Lear is a character in Shakespeare\'s play "King Lear." He is the king of Britain and the central figure of the play. Lear starts off as a powerful and proud ruler but descends into madness and despair as he faces betrayal and mistreatment from his daughters.'

## W and B Tracing

In [16]:
os.environ["WANDB_NOTEBOOK_NAME"] = "Python RAQA Example.ipynb"
wandb.init(project="RAQA Example")

wandb: Currently logged in as: megatron. Use `wandb login --relogin` to force relogin


In [17]:
retrieval_augmented_qa_pipeline = WandB_RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    wandb_project="LLM Visibility Example"
)

In [18]:
retrieval_augmented_qa_pipeline.run_pipeline("Who is Batman?",raqa_prompt, user_prompt)

"I don't know."

In [19]:
retrieval_augmented_qa_pipeline.run_pipeline("What happens to Cordelia?",raqa_prompt, user_prompt)

'Cordelia is dead. King Lear is holding her lifeless body in his arms, mourning her death.'